In [1]:
import pandas as pd
import requests
from bs4 import BeautifulSoup as bs
import time
from tqdm import tqdm

In [2]:
from selenium import webdriver

In [3]:
def musinsa_rank(category_num,page_num):
    url =  f"https://www.musinsa.com/ranking/best?period=now&age=ALL&mainCategory=00{category_num}&subCategory=&leafCategory=&price=&golf=false&kids=false&newProduct=false&exclusive=false&discount=false&soldOut=false&page={page_num}&viewType=small&priceMin=&priceMax="
    response = requests.get(url)
    html = bs(response.text, 'lxml')
    musinsa_rank_df = rbnl(html)

    return musinsa_rank_df

In [4]:
def rbnl(html):
    musinsa_rank_df = pd.DataFrame()

    #순위 뽑기
    rank_html = html.select('#goodsRankList > li > p')
    rank_no_list = []

    for i in rank_html:
        rank_no_list.append(i.string.strip())

    musinsa_rank_df['순위'] = rank_no_list

    #브랜드 이름 뽑기
    brand_html = html.select('#goodsRankList > li > div.li_inner > div.article_info > p.item_title > a')
    brand_list = []

    for i in brand_html:
        brand_list.append(i.string)



    #링크와 의류명 뽑기
    link_name_html = html.select('#goodsRankList > li > div.li_inner > div.article_info > p.list_info > a')
    link_list = []
    name_list = []

    for i in link_name_html:
        link_list.append(i['href'])
        name_list.append(i['title'])

    musinsa_rank_df['상품명']=name_list
    musinsa_rank_df['브랜드명']=brand_list
    musinsa_rank_df['url']=link_list

    #상세 페이지 크롤링
    musinsa_rank_df2 = specific_info(link_list)

    #데이터 프레임 옆으로 합치기
    musinsa_rank_df = pd.concat([musinsa_rank_df, musinsa_rank_df2], axis=1)

    return musinsa_rank_df

In [7]:
from pyasn1.compat.octets import null


def specific_info(link_list):
    headers={'User-Agent':'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/101.0.4951.67 Safari/537.36'}

    musinsa_rank_df = pd.DataFrame()
    part_num_list=[]
    sex_list=[]
    view_list=[]
    sales_list=[]
    like_list=[]
    review_list=[]
    star_list=[]


    for link in tqdm(link_list):
        #         print(link)
        response_1=requests.get(link,headers=headers)
        html_1=bs(response_1.text, 'lxml')

        #품번 리스트 생성
        part_num_html=html_1.select('#product_order_info > div.explan_product.product_info_section > ul > li:nth-child(1) > p.product_article_contents > strong')
        part_num=part_num_html[0].get_text().split('/')[-1].strip()
        part_num_list.append(part_num)

        #성별 리스트 생성
        sex_html=html_1.select("#product_order_info > div.explan_product.product_info_section > ul > li > p.product_article_contents > span.txt_gender")
        sex=sex_html[0].get_text().replace('\n',' ').strip()
        sex_list.append(sex)

        #셀레니움으로 원하는 데이터 가져오기
        driver = webdriver.Chrome('chromedriver.exe')
        driver.get(link)
        sel_html=driver.page_source
        html_2=bs(sel_html)

        #조회수 가져오기
        view_html=html_2.find_all("strong", {"id":"pageview_1m"})
        view=view_html[0].get_text()
        view_list.append(view)

        #누적 판매 가져오기
        sales_html=html_2.find_all("strong", {"id":"sales_1y_qty"})
        sales=sales_html[0].get_text()
        sales_list.append(sales)

        #좋아요 수 가져오기
        like_html=html_2.find_all("span", {"class": "prd_like_cnt"})
        like=like_html[0].get_text()
        like_list.append(like)

        #평점
        star_html=html_2.find_all("span", {"class": "prd-score__rating" })
        if len(star_html) == 0:
            star = "평점 0점"
        else:
            star=star_html[0].get_text()
        star_list.append(star)


        #구매 후기 개수 가져오기
        review_html=html_2.find_all("span", {"class": "prd-score__review-count"})
        if len(review_html) == 0:
            review = "후기 0개"
        else:
            review=review_html[0].get_text()
        review_list.append(review)


        #드라이버 닫아주기
        driver.close()

        #시간 추가
        time.sleep(0.01)


    musinsa_rank_df['조회수']=view_list
    musinsa_rank_df['좋아요']=like_list
    musinsa_rank_df['구매후기 개수']=review_list
    musinsa_rank_df['평점']=star_list
    musinsa_rank_df['누적판매량(1년)']=sales_list


    return musinsa_rank_df

In [8]:
category_num=input("Category 번호를 입력하세요: ")
page_num=input("Page 번호를 입력하세요: ")
final_df=musinsa_rank(category_num,page_num)

  0%|          | 0/90 [00:00<?, ?it/s]C:\Users\jpc10\AppData\Local\Temp\ipykernel_19664\3077604374.py:33: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  driver = webdriver.Chrome('chromedriver.exe')
100%|██████████| 90/90 [25:40<00:00, 17.12s/it] 


In [9]:
file_name = f"0319 musinsa_ranking_category{category_num}_page{page_num}.csv"
final_df.to_csv(file_name, index=False)
pd.read_csv(file_name)

,순위,상품명,브랜드명,url,조회수,좋아요,구매후기 개수,평점,누적판매량(1년)
0,1위,(시티보이) 오버핏 옥스포드 셔츠_SPYWD23C01,스파오,https://www.musinsa.com/app/goods/2272830?loc=...,22.4만 회 이상,"46,073","후기 17,548개 보기",4.9,5만 개 이상
1,2위,[23SS Ver.][PACKAGE] 시티보이 빅오버 워싱 데님 셔츠_4 Color,라퍼지스토어,https://www.musinsa.com/app/goods/3150516?loc=...,1.5만 회 이상,884,후기 3개 보기,4.7,100개 이상
2,3위,[MUSINSA EXCLUSIVE] 뉴욕 복싱 클럽 코튼 치노 셔츠 - 블랙,폴로 랄프 로렌,https://www.musinsa.com/app/goods/3072191?loc=...,1.5만 회 이상,314,후기 0개,평점 0점,100개 이상
3,4위,베이식 긴팔 티셔츠 2팩,무신사 스탠다드,https://www.musinsa.com/app/goods/2149254?loc=...,14.1만 회 이상,"57,235","후기 31,434개 보기",4.9,12만 개 이상
4,5위,레이어드 크루 넥 반팔 티셔츠_일반 기장 2팩,무신사 스탠다드,https://www.musinsa.com/app/goods/2122566?loc=...,8.7만 회 이상,"18,637","후기 14,555개 보기",4.9,5만 개 이상
...,...,...,...,...,...,...,...,...,...
85,86위,베이식 긴팔 티셔츠 [화이트],무신사 스탠다드,https://www.musinsa.com/app/goods/1144989?loc=...,4.1만 회 이상,"58,738","후기 66,755개 보기",4.8,4.7만 개 이상
86,87위,(23SS) 2 TONE ARCH TEE WHITE,예일,https://www.musinsa.com/app/goods/1841764?loc=...,4.5만 회 이상,"75,338","후기 12,873개 보기",4.8,6.4만 개 이상
87,88위,오버핏 그래피티 로고 후디-블랙,필루미네이트,https://www.musinsa.com/app/goods/2913659?loc=...,5만 회 이상,"10,355",후기 207개 보기,4.8,2천 개 이상
88,89위,Comfort Shirt 01 (Silver (Thin)),포터리,https://www.musinsa.com/app/goods/3043302?loc=...,1.5만 회 이상,641,후기 31개 보기,4.9,100개 이상
